In [ ]:
#Depends on the following installs. Installed from  conda-forge 
# as it handled conflicts better than from Cran
#install.packages("rgdal")
#install.packages('sf')
#install.packages("udunits2")
#install.packages("dggridR")

library(dggridR)
library(sf)
library(dplyr)

In [ ]:
# Read shp into sf object
shp <- "~/stations/station_points_combined.shp"
shp_out <- "~/stations/station_hex.shp"
shp_path <- "~/stations"
station.sf <- st_read(shp)

In [ ]:
# Inspect different DGGS by changing target area
dggs <- dgconstruct(projection="FULLER", aperture = 4, area=22)

# Read sf into hex grid approx. 30mx30m
dggs <- dgconstruct(area=0.0009)
points <- st_coordinates(station.sf)
temp.df <- as.data.frame(points)
temp.df$cell <- dgGEO_to_SEQNUM(dggs, temp.df$X, temp.df$Y)$seqnum

In [ ]:
# Aggregate by cell
counts <- temp.df %>% group_by(cell) %>% summarise(count=n())

In [ ]:
# Get cell boundary to save
dgcellstogrid(dggs, counts$cell,frame=FALSE,savegrid=shp_path)

# Get cell boundary to plot
grid <- dgcellstogrid(dggs, counts$cell,frame=TRUE,wrapcells=TRUE)
grid <- merge(grid, counts, by.x="cell", by.y="cell")

p <- ggplot() + 
     geom_polygon(data=grid, aes(x=long, y=lat, group=group, fill=count), alpha=0.4)    +
     geom_path   (data=grid, aes(x=long, y=lat, group=group), alpha=0.4, color="white") +
     scale_fill_gradient(low="blue", high="red")

In [ ]:
# Export to shp for plotting in arc
grid.sf <- st_as_sf(grid, coords = c("long", "lat"), crs=st_crs(station.sf))
st_write(grid.sf, shp_out)